In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

"""

重點流程：

- 掛載google drive
- 安裝套件
- 引入套件
- 建置主程序(含line_bot_api跟 webhookhandler)
- 建立主程序API介面接口(這樣才能接收用戶訊息)


* 建立handler收到用戶發送的 圖像(image) 訊息時，該如何處理的方法
* 建立handler收到用戶發送的 影像(video) 訊息時，該如何處理的方法
* 建立handler收到用戶發送的 音訊(audio) 訊息時，該如何處理的方法


- 最後三步驟需多練習

In [ ]:
"""

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

"""

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
"""
套件安裝
"""

!pip install line-bot-sdk flask flask-ngrok

In [ ]:
"""
引用套件
"""

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import LineBotApi, WebhookHandler

# 引用無效簽章錯誤
from linebot.exceptions import InvalidSignatureError

In [ ]:
"""
建置主程序

建置handler與 line_bot_api
"""

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("HPNN2U0LfdttfMl7b5yCkW/h9yzuxYOGW4NYDnJuBD4qXTY/UagvGJK3OKAnGwQwgA1tYG9lPNFt92N+j33osr3nQ59os0a/bD+usRNyUumvLs0IYnHTR6FkmhCAaQ8WkcKZq6a3AJlA4tfw9jxLtgdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("e20a1f0d6418dd3bebb261a9fb9374ab")

In [ ]:
"""
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
"""

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
"""

若收到圖片消息時，要如何處理

先回覆用戶文字消息，並從Line上將照片拿回。


概念：

當公司有包裹(圖片)需要領取時，郵局(LINE)會寄來憑單(event.message.id)通知公司(LINEBOT)記得去領取

公司(LINEBOT)

"""

# ***** 本章核心 *****

from linebot.models import MessageEvent, ImageMessage, TextSendMessage


# 當handler.遇到(訊息事件，訊息=圖像訊息)時，告知handler該如何處理

@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    
    # 業務透過郵局告知用戶，已收到圖像包裹取件通知，包裹的單號是XXX
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text="已收到用戶的寄來的圖片包裹，包裹單號ID是" + event.message.id))
    
    
    # 接著業務再依據領取憑單去郵局領取圖片包裹回來，然後存放在本地端 (如同業務憑單據去郵局領包裹)
    # 1. 先取包裹
    message_content = line_bot_api.get_message_content(event.message.id)
    
    # 2. 再放回公司的收發室(檔案室)
    with open(event.message.id + ".jpg", "wb") as fd:
      # 非文字檔案是二進位 => 需全部讀出 => 用for迴圈來取得檔案的每個片段
      for chunk in message_content.iter_content():
        fd.write(chunk)


In [ ]:
"""
若收到影片訊息時，同理
"""

from linebot.models import MessageEvent, TextSendMessage, VideoMessage

# 當handler.遇到(訊息事件, 訊息=影像訊息)時，告知handler該如何處理

@handler.add(MessageEvent, message=VideoMessage)
def handle_video_message(event):
  # 業務透過郵局告知用戶，已收到影像包裹取件通知，包裹的單號是XXX
  line_bot_api.reply_message(event.reply_token, TextSendMessage(text="已收到用戶寄來的影像包裹，包裹單號ID是" + event.message.id))

  # 再依據包裹ID從郵局取回用戶寄來的影片包裹，並放到儲藏室(收發室)
  message_content = line_bot_api.get_message_content(event.message.id)
  with open(event.message.id + ".mp4", "wb") as fd:
    for chunk in message_content.iter_content():
      fd.write(chunk)


In [ ]:

"""
若收到音訊訊息時，同理
"""

from linebot.models import MessageEvent, TextSendMessage, AudioMessage

# 當handler.遇到(訊息事件, 訊息=音訊訊息)時，告知handler該如何處理
@handler.add(MessageEvent, message=AudioMessage)
def handle_audio_message(event):
  # 業務透過郵局告知用戶，已收到音訊包裹取件通知，包裹的單號是XXX
  line_bot_api.reply_message(event.reply_token, TextSendMessage(text="已收到用戶寄來的影像包裹，包裹單號ID是" + event.message.id))

  # 再依據包裹ID從郵局取回用戶寄來的音訊包裹，並放到儲藏室(收發室)
  message_content = line_bot_api.get_message_content(event.message.id)
  with open(event.message.id + ".mp3", "wb") as fd:
    for chunk in message_content.iter_content():
      fd.write(chunk)


In [ ]:
# 主程序運行
app.run()